## Kaggle Setup

In [2]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"ll0406","key":"fa4830570075c294db90b927a8ca9fe8"}') 
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"ll0406","key":"fa4830570075c294db90b927a8ca9fe8"}') 
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73272 sha256=8911b6ae2009446a7479d721efff1bda806dc22810b0a4c92feef73b387bbf1a
  Stored in directory: /root/.cache/pip/wheels/d4/02/ef/3f8c8d86b8d5388a1d3155876837f1a1a3143ab3fc2ff1ffad
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.13
    Uninstalling kaggle-1.5.13:
      Successfully uninstalled kaggle-1.5.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached kaggle-1.5.8-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8
mkdir: cannot create directory ‘/root/.kaggle’

## Data Download

In [3]:
!kaggle competitions download -c m5-forecasting-accuracy
!mkdir '/content/data'

!unzip -qo 'm5-forecasting-accuracy.zip' -d '/content/data'

 96% 44.0M/45.8M [00:03<00:00, 20.9MB/s]
100% 45.8M/45.8M [00:03<00:00, 14.6MB/s]


### Library Import

In [4]:
import os
import gc
import time
import math
import datetime
from math import log, floor
from sklearn.neighbors import KDTree

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.utils import shuffle
from tqdm.notebook import tqdm as tqdm

import seaborn as sns
from matplotlib import colors
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import pywt
from statsmodels.robust import mad

import scipy
import statsmodels
from scipy import signal
from scipy.special import ndtri
import statsmodels.api as sm
from scipy.signal import butter, deconvolve
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

import warnings
warnings.filterwarnings("ignore")

## Load data


In [5]:
INPUT_DIR = './data'
calendar = pd.read_csv(f'{INPUT_DIR}/calendar.csv')
selling_prices = pd.read_csv(f'{INPUT_DIR}/sell_prices.csv')
sample_submission = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')
sales_train_val = pd.read_csv(f'{INPUT_DIR}/sales_train_validation.csv')

In [6]:
ids = sorted(list(set(sales_train_val['id'])))
d_cols = [c for c in sales_train_val.columns if 'd_' in c]
x_1 = sales_train_val.loc[sales_train_val['id'] == ids[2]].set_index('id')[d_cols].values[0]
x_2 = sales_train_val.loc[sales_train_val['id'] == ids[1]].set_index('id')[d_cols].values[0]
x_3 = sales_train_val.loc[sales_train_val['id'] == ids[17]].set_index('id')[d_cols].values[0]

## Inventory Preparation


In [22]:
EXAMPLE_SALES = x_1[-365:] # Use last year demand as example
print(len(EXAMPLE_SALES))

def gen_perfect_forecast(sales, forecast_period, leadtime):
    forecast = []
    for i in range(0, len(sales) - leadtime + 1):
        forecast_start = i
        forecast_end = min(len(sales), i + forecast_period)
        forecast.append(sales[forecast_start:forecast_end])
    
    return forecast

perfect_forecast = gen_perfect_forecast(EXAMPLE_SALES, 28, 5)


365


In [33]:
class SingleItemSimulate:
    def __init__(self, demand, forecast, leadtime, desired_alpha=0.99):
        self.demand = demand
        self.forecast = forecast
        self.leadtime = leadtime

        self.inventory_points = []
        self.net_inventory = []
        self.wip_inventory = []
        self.orders = []

        self.forecast_err = []
        self.forecast_err_variation = 0
        self.forecast_err_exp_smooth_alpha = 0.8

        self.sim_duration = len(demand) - 28 

        self.desired_service_level = desired_alpha
        self.newsvendor_multiplier = ndtri(desired_alpha)

    def get_lead_time_forecasted_demand(self, t):
        """Get forecasted_demand at time t
        """
        forecast_at_t = self.forecast[t]
        lead_time_demand = sum(forecast_at_t[0:self.leadtime])
        return lead_time_demand
    
    def run(self):
        for t in range(0, self.sim_duration):
            # Calculate leadtime forecasted demand
            lead_time_forecasted_demand = self.get_lead_time_forecasted_demand(t)

            # Update forecast error if t >= leadtime
            if t >= self.leadtime:
                actual_past_leadtime_demand_t = sum(self.demand[t-self.leadtime:t])
                forecasted_past_leadtime_demand_t = sum(self.forecast[t-self.leadtime][0:self.leadtime])
                err = actual_past_leadtime_demand_t - forecasted_past_leadtime_demand_t
                self.forecast_err.append(np.abs(err))

                # Update forecast err variation use exponential smoothing
                previous_forecast_err_variation = self.forecast_err_variation
                current_forecast_err_std = np.std(self.forecast_err)
                self.forecast_err_variation = self.forecast_err_exp_smooth_alpha * current_forecast_err_std + (1-self.forecast_err_exp_smooth_alpha) * previous_forecast_err_variation
            
            # Calculate inventory point
            yesterday_inventory_point = 0 if t < 1 else self.inventory_points[t-1]
            yesterday_order = 0 if t < 1 else self.orders[t-1]
            today_inventory_point = yesterday_inventory_point + yesterday_order - self.demand[t]
            self.inventory_points.append(today_inventory_point)

            # Calculate order
            order = lead_time_forecasted_demand + self.forecast_err_variation * self.newsvendor_multiplier - self.inventory_points[t]
            self.orders.append(order)

            # Calculate net inventory and WIP inventory
            yesterday_net_inventory = 0 if t < 1 else self.net_inventory[t-1]
            last_leadtime_order = 0 if t < self.leadtime else self.orders[t-self.leadtime]
            net_inventory = yesterday_net_inventory + last_leadtime_order - self.demand[t]
            
            yesterday_wip_inventory = 0 if t < 1 else self.wip_inventory[t-1]
            wip_inventory = yesterday_wip_inventory + yesterday_order - last_leadtime_order

            self.net_inventory.append(net_inventory)
            self.wip_inventory.append(wip_inventory)

    def calculate_performance(self):
        """Calculate the performance metrics for the inventory operation
        """
        holding_cost = 0
        for ni in self.net_inventory:
            if ni > 0:
                holding_cost += ni
        order_variance = np.var(self.orders)
        achieved_service_level = 1 - float((np.array(self.net_inventory) < 0).sum()) / len(self.net_inventory)

        return holding_cost, order_variance, achieved_service_level


In [30]:
print(EXAMPLE_SALES[0:5])
print(perfect_forecast[5-5][0:5])

[0 0 0 0 1]
[0 0 0 0 1]


In [35]:
simulate = SingleItemSimulate(EXAMPLE_SALES, perfect_forecast, 5, desired_alpha=0.999)
simulate.run()
simulate.calculate_performance()

(252.0, 17.367873275277585, 0.7299703264094956)